# Create Response Generator Dataset
This notebook creates the dataset for re-training the response generator model.

## Pre-requisites
A CSV file must be created that contains fields including input, search results, and output.

In [1]:
from citation_fetcher import Citation_Fetcher as cf
from query_parser import QueryParser as qp
import const
import numpy as np
import pandas as pd

In [20]:
RESPONSE_GEN_DS_PATH: str =  const.DATASETS_FOLDER + "response_gen_ground_truth.csv"
SQUAD_DS_PATH: str = const.DATASETS_FOLDER + "squad_ds_keyword_ground_truth.csv"
ULTRACHAT_LITE_PATH: str = const.DATASETS_FOLDER + "ultrachat_50k.csv"
REDDIT_DS_PATH: str = const.DATASETS_FOLDER + "reddit_ds.csv"

In [3]:
squad_ds: pd.DataFrame = pd.read_csv(SQUAD_DS_PATH)
squad_len = len(squad_ds)

/tmp/ipykernel_24785/1211425012.py:1: DtypeWarning: Columns (17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  squad_ds: pd.DataFrame = pd.read_csv(SQUAD_DS_PATH)


In [15]:
ultrachat_ds: pd.DataFrame = pd.read_csv(ULTRACHAT_LITE_PATH)
ultrachat_len = len(squad_ds)

In [21]:
reddit_ds: pd.DataFrame = pd.read_csv(REDDIT_DS_PATH)

In [11]:
rg_ds = squad_ds['question']
#rg_ds = rg_ds.rename(columns={'question': "input"})

In [12]:
rg_ds = pd.DataFrame(data={'input': rg_ds, 'query': None, 'search_result': None, 'output': None})

In [13]:
rg_ds.head()

,input,query,search result,output
0,When did Beyonce start becoming popular?,None,None,None
1,What areas did Beyonce compete in when she was...,None,None,None
2,When did Beyonce leave Destiny's Child and bec...,None,None,None
3,In what city and state did Beyonce grow up?,None,None,None
4,In which decade did Beyonce become famous?,None,None,None


In [17]:
ultrachat_ds.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,question,data,is_searchable
0,0,0,0,0,How do engineers design and test roller coaste...,['How do engineers design and test roller coas...,True
1,1,1,1,1,Can you discuss how religious motifs have been...,['Can you discuss how religious motifs have be...,True
2,2,2,2,2,How has the depiction of war in movies changed...,['How has the depiction of war in movies chang...,True
3,3,3,3,3,Analyze the use of mise-en-scène in Blade Runn...,"[""Analyze the use of mise-en-scène in Blade Ru...",True
4,4,4,4,4,What steps does Procter & Gamble take to promo...,['What steps does Procter & Gamble take to pro...,True


In [18]:
ultrachat_ds = ultrachat_ds.rename(columns={'question': 'input'})

In [ ]:
rg_ds  = pd.concat([rg_ds, ultrachat_ds['input']], ignore_index=True)

In [ ]:
i = 0
for row in rg_ds:
    print("Populating Search Result ", (int((i/len(rg_ds))*100)),"% (Sample ", (i+1), "/", len(rg_ds) + ")", 
                end="                   \r")
    user_input = row['input']
    query = qp.gen_query(user_input)
    json_out, result = cf.search_online(query)
    rg_ds[i, "search_result"] = result
    i+=1

In [22]:
reddit_ds.head()

,Unnamed: 0,0,1,2
0,0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
1,1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."
2,2,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und...","cool. I've been thinking of getting one, my ph..."
3,3,What kind of phone(s) do you guys have?,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?
4,4,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?,I love it. I can't think of anything I don't l...


In [ ]:
reddit_ds = reddit_ds.rename(columns={"0": "input"})

In [ ]:
rg_ds = pd.concat([rg_ds, reddit_ds['input']], ignore_index=True)

In [ ]:
rg_ds.to_csv(RESPONSE_GEN_DS_PATH)